# GPT2 Model

## Install necessary libraries

In [ ]:
!pip install transformers torch pandas scikit-learn

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.8 MB/s eta 0:00:00


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

In [ ]:
pip install tqdm

## Load necessary libraries

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from transformers import get_scheduler, TrainerCallback
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

## Load and Prepare the Data

In [ ]:
class EssayDataset(Dataset):
    def __init__(self, encodings, scores=None):
        self.encodings = encodings
        self.scores = scores

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.scores is not None:
            item['labels'] = torch.tensor(self.scores[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

### Train and test split with 20%

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

# Load your dataset
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Deep-Learning-Final-Project/data/train.csv')

# Split the data into training and validation sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['score'])
train_scores = train_df['score'].values
valid_scores = valid_df['score'].values

# Initialize the tokenizer
# Since GPT-2 does not use padding during its initial training; it processes sequences end-to-end. So we will use EOF(end-of-sentence) token to handle the data.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize essays
train_encodings = tokenizer(train_df['full_text'].tolist(), truncation=True, padding='max_length', max_length=512, pad_to_multiple_of=None)
test_encodings = tokenizer(valid_df['full_text'].tolist(), truncation=True, padding='max_length', max_length=512, pad_to_multiple_of=None)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
train_dataset = EssayDataset(train_encodings, train_scores)
val_dataset = EssayDataset(test_encodings, valid_scores)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

## Set up the Model

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model initialization
def model_init():
    model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=1)
    model.config.pad_token_id = tokenizer.eos_token_id
    return model

model = model_init().to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.cuda.amp import autocast
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.metrics import cohen_kappa_score

def calculate_qwk(y_true, y_pred):
    """Calculate the Quadratic Weighted Kappa (QWK)."""
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating", leave=False):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            with autocast():
                outputs = model(**inputs, labels=labels)
                loss = outputs.loss
                logits = outputs.logits.squeeze()

            total_loss += loss.item()
            # Round or convert predictions as necessary to ensure they are integers
            preds = logits.round().int()

            # Collect all predictions and labels for QWK calculation
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(val_loader)
    qwk = calculate_qwk(np.array(all_labels), np.array(all_preds))
    return avg_loss, qwk

## Train

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn

writer = SummaryWriter('runs/experiment_name')

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.cuda.amp import GradScaler, autocast
from transformers import GPT2ForSequenceClassification
from tqdm import tqdm
import torch.nn.functional as F
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_init().to(device)
# GradScaler for AMP
scaler = GradScaler()

optimizer = Adam(model.parameters(), lr=5e-4)

from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Training function
def train(model, train_loader, optimizer, scaler, device):
    model.train()
    total_loss = 0
    count = 0
    total_acc = 0
    for batch in tqdm(train_loader, desc="Training", leave=False):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
        out = outputs.logits.round().flatten()
        accuracy = len(out[out == labels])/len(out)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_acc += accuracy
        total_loss += loss.item()
        count += 1

    avg_loss = total_loss / len(train_loader)
    return avg_loss,total_acc/count

# Validation function
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    count = 0
    total_acc = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            with autocast():
                outputs = model(**inputs, labels=labels)
                loss = outputs.loss

            out = outputs.logits.round().flatten()
            accuracy = len(out[out == labels])/len(out)
            total_loss += loss.item()
            total_acc += accuracy
            count += 1
    avg_loss = total_loss / len(val_loader)

    return avg_loss, total_acc/count

# Main training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss,train_acc = train(model, train_loader, optimizer, scaler, device)
    val_loss,val_acc = validate(model, val_loader, device)

    scheduler.step()  # Update the learning rate
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f},Training Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

writer.close()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training:   0%|          | 0/866 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/10, Training Loss: 2.6519,Training Accuracy: 0.5262, Validation Loss: 0.3819, Validation Accuracy: 0.6046


Epoch 2/10, Training Loss: 0.3542,Training Accuracy: 0.6196, Validation Loss: 0.4850, Validation Accuracy: 0.5379


Epoch 3/10, Training Loss: 0.2412,Training Accuracy: 0.7076, Validation Loss: 0.4383, Validation Accuracy: 0.5655


Epoch 4/10, Training Loss: 0.1546,Training Accuracy: 0.8044, Validation Loss: 0.4517, Validation Accuracy: 0.5776


...

Epoch 6/10, Training Loss: 0.0741,Training Accuracy: 0.9173, Validation Loss: 0.4876, Validation Accuracy: 0.5523


Epoch 7/10, Training Loss: 0.0686,Training Accuracy: 0.9236, Validation Loss: 0.4752, Validation Accuracy: 0.5573


Epoch 8/10, Training Loss: 0.0574,Training Accuracy: 0.9354, Validation Loss: 0.5130, Validation Accuracy: 0.5326


Epoch 9/10, Training Loss: 0.0512,Training Accuracy: 0.9461, Validation Loss: 0.4899, Validation Accuracy: 0.5679


Epoch 10/10, Training Loss: 0.0440,Training Accuracy: 0.9546, Validation Loss: 0.4884, Validation Accuracy: 0.5621
